In [1]:
import os
#import scipy
import pandas as pd
import numpy as np
from numpy import cov

from scipy import linalg
from sklearn import preprocessing
import seaborn as sns
sns.set(color_codes = True)
import matplotlib.pyplot as plt

%matplotlib inline

In [11]:
df = pd.read_csv("GK.csv")
# df = pd.read_csv("C:/Users/Ranja.Sarkar/Desktop/PAM/GK.csv", header = None)

C:\Users\Ranja.Sarkar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df['datetime'] = pd.to_datetime(df['timestamp'])
df = df.set_index('datetime')
df = df.drop('timestamp', axis = 1)
df = df.sort_index()

In [15]:
#df.head(5)
## df.to_csv('indexed_sorted_GK.csv')
## df = pd.read_csv("C:/Users/Ranja.Sarkar/Desktop/PAM/indexed_sorted_GK.csv")

In [77]:
## df.plot(figsize=(16,8))
## Jul2017 - Nov2018
# from datetime import datetime
## pd.date_range('2017','2018')

In [5]:
## Scale data
scaler = preprocessing.MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns, index = df.index)

In [10]:
#df.tail(3)
## df.plot(figsize=(16,8))

In [6]:
# Define train/test data
train = df['2017-07-01 00:10:00':'2018-07-30 00:30:00']
test = df['2018-07-30 00:40:00':]

In [7]:
train.shape, test.shape

((48987, 170), (0, 170))

In [8]:
# Compress data (sensor readings) to 2 main PCs here
from sklearn.decomposition import PCA
pca = PCA(n_components = 2, svd_solver = 'full')

In [9]:
train_PCA = pca.fit_transform(train)
train_PCA = pd.DataFrame(train_PCA)
train_PCA.index = train.index

test_PCA = pca.transform(test)
test_PCA = pd.DataFrame(test_PCA)
test_PCA.index = test.index

ValueError: Found array with 0 sample(s) (shape=(0, 170)) while a minimum of 1 is required.

In [87]:
# define functions (covariance matrix, inverse covariance or concentration matrix representing partial/pairwise correlations)
def cov_matrix(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            return covariance_matrix, inv_covariance_matrix
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite")
    else:
        print("Error: Covariance Matrix is not positive definite")

In [97]:
# define functions (Mahalanobis distance metric)
def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    inv_cov_matrix = inv_covariance_matrix
    vars_mean = mean_distr
    diff = data - vars_mean
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))
    return md

In [89]:
# define functions (detect outlier)
def MD_detectOutliers(dist, extreme = False, verbose = False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    outliers = []
    for i in range(len(dist)):
        if dist[i] >= threshold:
            outliers.append(i)  # index of the outlier
    return np.array(outliers)

In [90]:
# define functions (threshold value)
def MD_threshold(dist, extreme = False, verbose = False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    return threshold

In [91]:
# define functions (positive definite check for matrix)
def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

In [105]:
# define train/test data from the 2 main PCs
data_train = np.array(train_PCA.values)
data_test = np.array(test_PCA.values)

In [111]:
# mean value for input variables in the training dataset
mean_distr = data_train.mean(axis = 0)
covariance_matrix, inv_cov_matrix = cov_matrix(data_train,verbose = False)

In [112]:
dist_train = MahalanobisDist(inv_cov_matrix, mean_distr, data_train, verbose = False)
dist_test = MahalanobisDist(inv_cov_matrix, mean_distr, data_test, verbose = False)
threshold = MD_threshold(dist_train, extreme = True)